In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error, r2_score
import re
import warnings
warnings.filterwarnings('ignore')

In [51]:
def clean_data(input_file='Data/ToxicityPhylogeneticChemical.csv'):    
    all_data = pd.read_csv(input_file)
    
    # Function to extract just the number
    def extract_number(endpoint_str):
        if pd.isna(endpoint_str):
            return None
        # Remove everything except numbers and decimal points
        match = re.search(r'(\d+\.?\d*)', str(endpoint_str))
        return float(match.group(1)) if match else None

    # Function to extract units (mg/l or mg/kg)
    def extract_units(endpoint_str):
        if pd.isna(endpoint_str):
            return None
        endpoint_str = str(endpoint_str).lower()
        if 'mg/l' in endpoint_str:
            return 'mg/l'
        elif 'mg/kg' in endpoint_str:
            return 'mg/kg'
        else:
            return None

    # Apply the functions
    all_data['Endpoint Val'] = all_data['Endpoint Value'].apply(extract_number)
    all_data['Endpoint Units'] = all_data['Endpoint Value'].apply(extract_units)
    all_data = all_data.drop(columns=['Endpoint Value'])

    # Filter and clip endpoint values
    all_data = all_data[all_data['Endpoint Val'] > 0.005]
    all_data['Endpoint Val'] = all_data['Endpoint Val'].clip(lower=0.01, upper=5000)

    # Hierarchical encoding of phylogenetic data
    class_features = pd.get_dummies(all_data['class'], prefix='class')
    order_features = pd.get_dummies(all_data['order'], prefix='order') 
    family_features = pd.get_dummies(all_data['family'], prefix='family')
    genus_features = pd.get_dummies(all_data['genus'], prefix='genus')

    hierarchical_features = pd.concat([
        class_features,
        order_features, 
        family_features,
        genus_features
    ], axis=1)

    all_data = pd.concat([all_data, hierarchical_features], axis=1)
    
    # Drop unnecessary columns
    columns_to_drop = ['Unnamed: 0', 'Common Name', 'Range', 'Sex', 'Chemical', 
                      'ChemicalName', 'Sample Size', 'Tox Exposure', 'species', 
                      'class', 'order', 'family', 'genus']
    # Only drop columns that exist
    columns_to_drop = [col for col in columns_to_drop if col in all_data.columns]
    all_data = all_data.drop(columns=columns_to_drop)

    # Normalizing Chemical Data
    all_data['Lipinski_Violations'] = (
        (all_data['MolecularWeight'] > 500).astype(int) +
        (all_data['XLogP'] > 5).astype(int) +
        (all_data['HBondDonorCount'] > 5).astype(int) +
        (all_data['HBondAcceptorCount'] > 10).astype(int)
    )

    all_data['Total_HBonds'] = all_data['HBondDonorCount'] + all_data['HBondAcceptorCount']
    
    # Standardize molecular features
    scaler = StandardScaler()
    all_data['MolecularWeight'] = scaler.fit_transform(all_data[['MolecularWeight']])
    all_data['TPSA'] = StandardScaler().fit_transform(all_data[['TPSA']])
    all_data['XLogP'] = all_data['XLogP'].fillna(all_data['XLogP'].median())

    # Handling Tox Exposure Duration
    arr = ['Single dose','single', 'single dose', 'Single exposure']
    all_data['Tox Exposure Duration'] = all_data['Tox Exposure Duration'].replace(arr, 0.1)
    all_data['Tox Exposure Duration'] = all_data['Tox Exposure Duration'].fillna(0.1)
    all_data['Tox Exposure Duration'] = all_data['Tox Exposure Duration'].astype('float64')

    # Handling Tox Exposure Technique
    all_data['Tox Exposure Technique'] = all_data['Tox Exposure Technique'].fillna('unknown')
    technique_features = pd.get_dummies(all_data['Tox Exposure Technique'], prefix='technique')
    all_data = pd.concat([all_data, technique_features], axis=1)
    all_data = all_data.drop(columns=['Tox Exposure Technique'])

    # Encoding Life Cycle Stage
    lifecycle_features = pd.get_dummies(all_data['Life Cycle Stage'], prefix='stage')
    all_data = pd.concat([all_data, lifecycle_features], axis=1)
    all_data = all_data.drop(columns=['Life Cycle Stage'])

    # Log transforming endpoint value
    all_data['log_endpoint_val'] = np.log(all_data['Endpoint Val'])
    all_data = all_data.drop(columns=['Endpoint Val'])
    
    return all_data

In [74]:
class ModelPipeline:
    def __init__(self):
        self.endpoint_type = RandomForestClassifier(random_state=42)
        self.endpoint_val = RandomForestRegressor(random_state=42)
        self.endpoint_units = RandomForestClassifier(random_state=42)

    def train_data(self, df):
        training_data, testing_data = train_test_split(
            df,
            test_size=0.25, 
            random_state=42
        )
        
        endpoint_encoder = LabelEncoder()
        units_encoder = LabelEncoder()
        
        training_data['endpoint_type_encoded'] = endpoint_encoder.fit_transform(training_data['Endpoint Description'])
        training_data['units_encoded'] = units_encoder.fit_transform(training_data['Endpoint Units'])

        training_technique_cols = [col for col in training_data.columns if col.startswith('technique_')]
        training_stage_cols = [col for col in training_data.columns if col.startswith('stage_')]
        training_class_cols = [col for col in training_data.columns if col.startswith('class_')]

        model1_features = ['Tox Exposure Duration'] + training_technique_cols
        model2_features = training_stage_cols + training_class_cols + training_technique_cols + ['MolecularWeight', 'XLogP', 'TPSA', 'HBondDonorCount', 'HBondAcceptorCount', 'Lipinski_Violations', 'Total_HBonds'] + ['Tox Exposure Duration', 'endpoint_type_encoded']
        model3_features = ['endpoint_type_encoded'] + training_technique_cols + training_class_cols


        testing_data['endpoint_type_encoded'] = endpoint_encoder.transform(testing_data['Endpoint Description'])
        testing_data['units_encoded'] = units_encoder.transform(testing_data['Endpoint Units'])

        #Model 1 training
        model1_training_data = training_data.copy()
        model1_testing_data = testing_data.copy()
        
        X1_train = model1_training_data[model1_features]
        y1_train = model1_training_data['endpoint_type_encoded']
        X1_test = model1_testing_data[model1_features]
        y1_test = model1_testing_data['endpoint_type_encoded']
        
        # Train model (no scaling needed for Random Forest)
        self.endpoint_type.fit(X1_train, y1_train)
        y1_pred = self.endpoint_type.predict(X1_test)
        model1_accuracy = accuracy_score(y1_test, y1_pred)

        print(f"Model 1 Accuracy: {model1_accuracy:.3f}")
        print(f"Evaluated on ALL {len(y1_test)} test samples")
        
        # Show classification report for available classes
        try:
            print("Classification Report:")
            print(classification_report(y1_test, y1_pred, target_names=endpoint_encoder.classes_))
        except:
            print("⚠️  Some classes may be missing from test set")



        #Model 2 training
        model2_training_data = training_data.copy()
        model2_testing_data = testing_data.copy()

        X2_train = model2_training_data[model2_features]
        y2_train = model2_training_data['log_endpoint_val']
        X2_test = model2_testing_data[model2_features]
        y2_test = model2_testing_data['log_endpoint_val']

        self.endpoint_val.fit(X2_train, y2_train)
        y2_pred = self.endpoint_val.predict(X2_test)
        model2_r2 = r2_score(y2_test, y2_pred)
        model2_rmse = np.sqrt(mean_squared_error(y2_test, y2_pred))
        print(f"Testing R2 score: {model2_r2:.2f}" )
        print(f"Testing RSME score: {model2_rmse:.2f}" )
        
        #Model 3 training
        model3_training_data = training_data.copy()
        model3_testing_data = testing_data.copy()

        X3_train = model3_training_data[model3_features]
        y3_train = model3_training_data['units_encoded']
        X3_test = model3_testing_data[model3_features]
        y3_test = model3_testing_data['units_encoded']

        self.endpoint_units.fit(X3_train, y3_train)
        y3_pred = self.endpoint_units.predict(X3_test)
        model3_accuracy = accuracy_score(y3_test, y3_pred)

        print(f"Model 3 Accuracy: {model3_accuracy:.3f}")
        print(f"Evaluated on ALL {len(y3_test)} test samples")

        # Show classification report for available classes
        try:
            print("Classification Report:")
            print(classification_report(y3_test, y3_pred, target_names=units_encoder.classes_))
        except:
            print("⚠️  Some classes may be missing from test set")


In [75]:
def main():
    cleaned_data = clean_data()
    model = ModelPipeline()
    model.train_data(cleaned_data)

if __name__ == "__main__":
    main()

Model 1 Accuracy: 0.971
Evaluated on ALL 34 test samples
Classification Report:
              precision    recall  f1-score   support

        LC50       0.93      1.00      0.96        13
        LD50       1.00      0.95      0.98        21

    accuracy                           0.97        34
   macro avg       0.96      0.98      0.97        34
weighted avg       0.97      0.97      0.97        34

Testing R2 score: 0.74
Testing RSME score: 1.84
Model 3 Accuracy: 1.000
Evaluated on ALL 34 test samples
Classification Report:
              precision    recall  f1-score   support

       mg/kg       1.00      1.00      1.00        26
        mg/l       1.00      1.00      1.00         8

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34

